# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [1]:
#!pip install -qU ragas==0.2.10

In [2]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

In [3]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [4]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [5]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [6]:
!mkdir data

In [7]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31427    0 31427    0     0  46949      0 --:--:-- --:--:-- --:--:-- 46905--:--:-- --:--:--     0


In [8]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70286    0 70286    0     0   228k      0 --:--:-- --:--:-- --:--:--  228k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [9]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [10]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [11]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [12]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [13]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 14, relationships: 72)

We can save and load our knowledge graphs as follows.

In [14]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 14, relationships: 72)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [15]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [16]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

1. The `SingleHopSpecificQuerySynthesizer` generates queries that are specific to a particular single source of information provided in the knowledge graph.
2. The `MultiHopAbstractQuerySynthesizer` generates queries that require referencing multiple sources in the knowledge graph, and require general understanding or interpretation of the data rather than a specific piece of information.
3. Similarly, the `MultiHopSpecificQuerySynthesizer` generates queries that require referencing multiple sources in the knowledge graph, but these queries are specific, meaning they require understanding multiple pieces of information and synthesizing them to find a specific answer.


Finally, we can use our `TestSetGenerator` to generate our testset!

In [17]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,what we learn about LLMs in 2024 and how they ...,[Code may be the best application The ethics o...,"In 2024, we learned that large language models...",single_hop_specifc_query_synthesizer
1,How does the use of Python in programming rela...,[Based Development As a computer scientist and...,The grammar rules of programming languages lik...,single_hop_specifc_query_synthesizer
2,Whaat were the key developments in AI since th...,[Simon Willison’s Weblog Subscribe Stuff we fi...,The academic field of Artificial Intelligence ...,single_hop_specifc_query_synthesizer
3,what stanford alpaca do in ai?,[easy to follow. The rest of the document incl...,Stanford Alpaca is mentioned in the context of...,single_hop_specifc_query_synthesizer
4,Was the best LLM trained in China for less tha...,[Prompt driven app generation is a commodity a...,The context mentions a question about whether ...,single_hop_specifc_query_synthesizer
5,What are the legal implications of AI in data ...,[<1-hop>\n\neasy to follow. The rest of the do...,"The legal implications of AI, particularly in ...",multi_hop_abstract_query_synthesizer
6,How does the concept of 'Vibes Based Developme...,[<1-hop>\n\nBased Development As a computer sc...,The concept of 'Vibes Based Development' refle...,multi_hop_abstract_query_synthesizer
7,How do AGI robustness and AI ethics intersect ...,[<1-hop>\n\neasy to follow. The rest of the do...,AGI robustness and AI ethics intersect in the ...,multi_hop_abstract_query_synthesizer
8,How has Meta contributed to the development an...,[<1-hop>\n\nCode may be the best application T...,Meta has significantly contributed to the deve...,multi_hop_specific_query_synthesizer
9,How do the challenges of gullibility and the l...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,The challenges of gullibility and the lack of ...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [18]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/26 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [19]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Wht is Anthropic?,[Code may be the best application The ethics o...,Anthropic is one of the organizations that hav...,single_hop_specifc_query_synthesizer
1,How does the use of Python in programming rela...,[Based Development As a computer scientist and...,"The context highlights that writing code, part...",single_hop_specifc_query_synthesizer
2,What were the significant developments in Larg...,[Simon Willison’s Weblog Subscribe Stuff we fi...,"In 2023, Large Language Models (LLMs) experien...",single_hop_specifc_query_synthesizer
3,Wht r the implicashuns of ChatGPT on human jobs?,[easy to follow. The rest of the document incl...,People have certainly lost work to AI models l...,single_hop_specifc_query_synthesizer
4,How do the criticisms of Large Language Models...,[<1-hop>\n\nPrompt driven app generation is a ...,The criticisms of Large Language Models (LLMs)...,multi_hop_abstract_query_synthesizer
5,How do the criticisms of large language models...,[<1-hop>\n\nPrompt driven app generation is a ...,The criticisms of large language models (LLMs)...,multi_hop_abstract_query_synthesizer
6,How do criticisms of LLMs relate to their ethi...,[<1-hop>\n\nPrompt driven app generation is a ...,Criticisms of LLMs are closely tied to concern...,multi_hop_abstract_query_synthesizer
7,How do the criticisms of large language models...,[<1-hop>\n\nPrompt driven app generation is a ...,Criticisms of large language models (LLMs) are...,multi_hop_abstract_query_synthesizer
8,In the context of Simon Willison's reflections...,[<1-hop>\n\neasy to follow. The rest of the do...,Simon Willison's reflections on AI advancement...,multi_hop_specific_query_synthesizer
9,How does the development of Llama 3.3 reflect ...,[<1-hop>\n\neasy to follow. The rest of the do...,"The development of Llama 3.3, which involved f...",multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [23]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years!"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [24]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [25]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [27]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [28]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

In [29]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [30]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [31]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Finally, we can set-up our RAG LCEL chain!

In [32]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [33]:
rag_chain.invoke({"question" : "What are Agents?"})

'Agents are a vague term that generally refers to AI systems thought to be capable of acting on behalf of individuals. There are two main interpretations: some see agents as systems that perform tasks like a travel agent, while others think of them as LLMs (large language models) that utilize tools in a loop to solve problems. However, the term lacks a clear and widely understood definition, leading to frustration and skepticism about their current utility and real-world implementation.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4o as our evaluation LLM for our base Evaluators.

In [34]:
eval_llm = ChatOpenAI(model="gpt-4o")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [35]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`: A `qa` type evaluator (question-answer) that assesses the quality of the answers to the questions by comparing the model's output with reference answers.
- `labeled_helpfulness_evaluator`: A `criteria` type evaluator used to assess the helpfulness of the response by providing the context of the question, answer, and reference answer and asking if the answer is helpful.
- `dope_or_nope_evaluator`: Another `criteria` type evaluator used to assess "dopeness" of the response. Dopeness being a less objective quality and more about style.

Answers above in-line

## LangSmith Evaluation

In [36]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'sunny-paint-75' at:
https://smith.langchain.com/o/fc40e18c-4e2c-4d6d-b6fb-30dadf24f85b/datasets/89a5e2f9-e2ab-4321-bc22-a3be89e03cfb/compare?selectedSessions=b2120a18-aea8-42d1-89b4-ebaf112e3479




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How does the fine-tuning process of Meta's Lla...,The fine-tuning process of Meta's Llama 3.3 us...,None,"The fine-tuning process of Meta's Llama 3.3, w...",1,1,1,2.961565,4a68365e-27b1-423a-ac24-cafb6159987f,66cbc4b7-697b-49d0-94b0-e66918317820
1,How do the challenges of evaluating LLMs and t...,I don't know.,None,"The challenges of evaluating LLMs, such as the...",0,0,0,1.495233,b670fcea-fdb6-4398-a763-1c8259ff75e1,fa590663-16dd-484c-bb36-f8720234ac50
2,How does the development of Llama 3.3 reflect ...,I don't know.,None,"The development of Llama 3.3, which involved f...",0,0,0,1.267137,eaa44290-d723-4e84-86db-9006d26f97ca,4cf14773-08de-4898-bd3b-79c923b43898
3,In the context of Simon Willison's reflections...,"In 2023, Large Language Models (LLMs) were not...",None,Simon Willison's reflections on AI advancement...,1,0,1,3.387900,c98ecc77-b44d-4200-b319-bd9806ab5aab,e2d29bf6-c998-4510-b426-ff5a769911c4
4,How do the criticisms of large language models...,The criticisms of large language models (LLMs)...,None,Criticisms of large language models (LLMs) are...,1,0,0,4.375113,e0e2525b-d04d-4758-8846-b89356495696,ebfd0507-d1ff-4b21-9071-6a4af0de103a
5,How do criticisms of LLMs relate to their ethi...,Criticisms of LLMs are closely related to thei...,None,Criticisms of LLMs are closely tied to concern...,1,0,0,3.059429,aed7685f-d135-4225-833e-a2673ef0ce79,b5697e81-74c4-4e12-9d7c-a7032c23d2ff
6,How do the criticisms of large language models...,I don't know.,None,The criticisms of large language models (LLMs)...,0,0,0,1.146683,55b48076-9312-420d-bc4c-50cc573bee1e,3bb5a4e6-79f5-452d-94c4-18d2dfa29a1f
7,How do the criticisms of Large Language Models...,The criticisms of Large Language Models (LLMs)...,None,The criticisms of Large Language Models (LLMs)...,1,0,0,2.462343,46077d1b-f419-42e3-8597-628142442add,bed8ab35-471b-46c7-ac59-2da2822b1016
8,Wht r the implicashuns of ChatGPT on human jobs?,I don't know.,None,People have certainly lost work to AI models l...,0,0,0,1.171627,c331b01a-a142-45a9-a56a-fdb1d9701e48,a667f115-b41b-477a-99b8-eaa8bb8d4ef5
9,What were the significant developments in Larg...,"In 2023, significant developments in Large Lan...",None,"In 2023, Large Language Models (LLMs) experien...",1,1,0,3.661179,98548a11-4799-430b-8af9-a58f156fbd8b,0af36bbe-0ca7-4155-8cfb-09797f911392


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [37]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [38]:
rag_documents = docs

In [39]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

- Adjusting chunk size can optimize or de-optimize depedning on a number of things. A larger chunk provides more context for the model when generating responses, but a larger chunk could also contain too much irrelevant information. On the other hand, smaller chunks might leave out important context, but samller chunks are faster to process. Ideally, we only want to provide the exact context that the model needs in order to generate a good response; this is the concept of the "golden chunk". 

In [40]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

- We are modifying embedding model to a larger one (`text-embedding-3-small` to `text-embedding-3-large`). The larger embedding models have higher dimensionality, meaning there are more relationship vectors. This results in the ability to capture more advanced semantics more accurately, but at the cost of performance and resource usage.

In [41]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [42]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [43]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [44]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

'Agents, huh? Well, that term\'s a bit of a slippery fish! People generally use it to refer to AI systems that supposedly act on your behalf—like a digital travel agent. But there’s a whole buffet of definitions out there, with some thinking about LLMs that can run tools in a loop to solve problems. But the kicker is, there\'s still a ton of confusion about what "agents" really mean. So, you could say they\'re kinda like the mythical creatures of the AI world—always being talked about but still not really in production. Without clarity, they’re just hanging in the background, waiting to become real!'

Finally, we can evaluate the new chain on the same test set!

In [45]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'brief-suit-61' at:
https://smith.langchain.com/o/fc40e18c-4e2c-4d6d-b6fb-30dadf24f85b/datasets/89a5e2f9-e2ab-4321-bc22-a3be89e03cfb/compare?selectedSessions=47a41c7f-edfc-4575-8bf3-3ffeb0d6c59e




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How does the fine-tuning process of Meta's Lla...,Meta's Llama 3.3 fine-tuning using over 25 mil...,None,"The fine-tuning process of Meta's Llama 3.3, w...",1,1,1,4.683827,4a68365e-27b1-423a-ac24-cafb6159987f,bf585462-1454-4054-a9e3-9ba16356edc2
1,How do the challenges of evaluating LLMs and t...,"Yo, the challenges of evaluating LLMs and thei...",None,"The challenges of evaluating LLMs, such as the...",1,0,1,4.148033,b670fcea-fdb6-4398-a763-1c8259ff75e1,1f615563-e9a3-44be-9755-96216ebb5aae
2,How does the development of Llama 3.3 reflect ...,"Yo, the development of Llama 3.3 shows us that...",None,"The development of Llama 3.3, which involved f...",1,1,1,3.733885,eaa44290-d723-4e84-86db-9006d26f97ca,d8777994-79ad-4774-b98a-0ed19521f061
3,In the context of Simon Willison's reflections...,"Yo, let’s break it down! In 2023, Simon Willis...",None,Simon Willison's reflections on AI advancement...,1,0,1,3.481312,c98ecc77-b44d-4200-b319-bd9806ab5aab,0ce74c1a-be98-411b-8ca4-f4925fdba8b7
4,How do the criticisms of large language models...,"Yo, the criticisms of LLMs hit hard on environ...",None,Criticisms of large language models (LLMs) are...,1,0,1,3.498575,e0e2525b-d04d-4758-8846-b89356495696,acde7003-7975-4591-a698-332e989bba04
5,How do criticisms of LLMs relate to their ethi...,"Yo, so the deal with criticisms of LLMs is tha...",None,Criticisms of LLMs are closely tied to concern...,1,0,1,4.028134,aed7685f-d135-4225-833e-a2673ef0ce79,045f218c-1b98-4a75-9964-11f2f4e90b47
6,How do the criticisms of large language models...,"Yo, it's a wild scene out there! The criticism...",None,The criticisms of large language models (LLMs)...,1,1,1,3.958434,55b48076-9312-420d-bc4c-50cc573bee1e,13f0872d-76b5-44d3-86ad-859581a1b852
7,How do the criticisms of Large Language Models...,"Yo, the criticisms of LLMs are totally intertw...",None,The criticisms of Large Language Models (LLMs)...,1,0,1,3.541912,46077d1b-f419-42e3-8597-628142442add,f0dea31f-c4f7-4170-9308-7ec3ae9a8245
8,Wht r the implicashuns of ChatGPT on human jobs?,I don't know.,None,People have certainly lost work to AI models l...,0,0,0,1.024105,c331b01a-a142-45a9-a56a-fdb1d9701e48,65ff0f6d-1cdd-4269-b368-5d3e3777dc7b
9,What were the significant developments in Larg...,"Alright, let me break it down for you! In 2023...",None,"In 2023, Large Language Models (LLMs) experien...",1,1,1,3.796179,98548a11-4799-430b-8af9-a58f156fbd8b,e84fc1b6-67d0-4559-a72c-8b68414665b5


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

As we can see below: correctness, dopeness, and helpfulness all increased significantly.
1) Correctness improved because we used the larger embedding model increasing semantic accuracy
2) Helpfulness improved  because we increase the chunk size, giving more context to the LLM to give a better answer.
2) Dopeness improved because we explicitly stated to be dope in the prompt: `You must answer the questions in a dope way, be cool!`

![Diff Image](langsmith_experiments.png)

